# **Titanic Prediction**

### **Import necessary libraries**

In [102]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

### **Load the dataset**

In [103]:
titanic_data = pd.read_csv('train.csv')

### **Display the first few rows of the dataset**

In [104]:
titanic_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## **Data preprocessing**

### **Define categorical and numerical features**

In [105]:
categorical_features = ['Sex', 'Embarked']
numerical_features = ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']

### **Create preprocessing pipeline for numerical features**

In [106]:
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median'))
])

### **Create preprocessing pipeline for categorical features**

In [107]:
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(drop='first'))
])

### **Combine preprocessing pipelines using ColumnTransformer**

In [108]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ])

### **Define the model pipeline**

In [109]:
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(n_estimators=100, random_state=42))
])

### **Split the data into features and target variable**

In [110]:
X_train = titanic_data.drop('Survived', axis=1)
y_train = titanic_data['Survived']

### **Model training**

In [111]:
model.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median'))]),
                                                  ['Pclass', 'Age', 'SibSp',
                                                   'Parch', 'Fare']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(drop='first'))]),
                                                  ['Sex', 'Embarked'])])),
                ('classifier', RandomForestClassifier(random_state=42))])

## **Model evaluation**

### **Predict on the test set**

In [112]:
X_test = pd.read_csv('test.csv')

In [113]:
y_pred = model.predict(X_test)

In [114]:
len(y_pred)

418

### **Calculate accuracy**

In [115]:
y_test = pd.read_csv('gender_submission.csv', usecols=['Survived'])

In [116]:
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.8205741626794258


### **Generate classification report**

In [117]:
print("Classification Report:")
print(classification_report(y_test, y_pred))

Classification Report:
              precision    recall  f1-score   support

           0       0.86      0.86      0.86       266
           1       0.75      0.76      0.75       152

    accuracy                           0.82       418
   macro avg       0.81      0.81      0.81       418
weighted avg       0.82      0.82      0.82       418



In [118]:
id_df = X_test[['PassengerId']]

y_pred_df = pd.DataFrame({'Survived': y_pred})

y_pred_final = pd.concat([id_df, y_pred_df], axis=1)

In [119]:
len(y_pred_final)

418

In [120]:
y_pred_final.to_csv('Predictions.csv',index=False)

In [121]:
y_pred_final

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,1
4,896,0
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [122]:
df1 = pd.read_csv('gender_submission.csv')

df2 = pd.read_csv('Predictions.csv')

if not df1.iloc[:, 0].equals(df2.iloc[:, 0]):
    df2.iloc[:, 0] = df1.iloc[:, 0]

    df2.to_csv('Output.csv', index=False)